# Convert TensroFlow saved_model to [TensorRT](https://github.com/NVIDIA/TensorRT/tree/main/samples/python/tensorflow_object_detection_api) Engine

define model location

In [ ]:
workdir = '/home/edurso/workspace/obj-detect/models/trained_model/'
mname = 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8'

download `trained_model` from google drive to `workdir`

## Installation & Environment Configuration

install tfod api

In [ ]:
import os
import pathlib

os.chdir('..')

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
%%bash

# update pip
pip3 install --upgrade pip

# check opencv dependency versions
pip3 uninstall opencv-python-headless==4.5.5.62 
pip3 install opencv-python-headless==4.1.2.30
pip3 install onnx-graphsurgeon==0.3.10 --index-url https://pypi.ngc.nvidia.com

# compile protos and install object_detection
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python3 -m pip install . --quiet

In [ ]:
!pip show object-detection

get tensorrt utility files

In [ ]:
url = 'https://raw.githubusercontent.com/NVIDIA/TensorRT/main/samples/python/tensorflow_object_detection_api/{}.py'
files = ['build_engine', 'create_onnx', 'infer', 'compare_tf', 'infer', 'onnx_utils', 'visualize']
os.chdir('../scripts')
for file in files:
  furl = url.format(file)
  !wget {furl}

## Create TensorRT Engine

In [ ]:
os.chdir('../scripts')
!python3 create_onnx.py \
    --pipeline_config {workdir}pipeline.config \
    --saved_model {workdir}saved_model \
    --onnx {workdir}model.onnx

In [ ]:
os.chdir('../scripts')
!python3 build_engine.py \
    --onnx {workdir}model.onnx \
    --engine {workdir}engine.trt \
    --precision fp16

## Test TensorRT Engine

In [ ]:
!trtexec \
    --loadEngine={workdir}engine.trt \
    --useCudaGraph --noDataTransfers \
    --iterations=100 --avgRuns=100

In [ ]:
os.chdir('../scripts')
!python3 infer.py \
    --engine {workdir}engine.trt \
    --input ../img \
    --output {workdir}/output \
    --preprocessor fixed_shape_resizer \
    --labels {workdir}label_map.txt